#  Bronze Layer Ingestion Notebook (Workflow Ready)
This notebook simulates production ingestion of raw Synthea data from cloud storage into Delta Lake Bronze tables. 

In a real production system, this notebook would be triggered automatically when a new file arrives in an AWS S3 bucket.

---

##  Conceptual Workflow (S3 → Databricks Job Trigger)

1. **Raw data file uploaded to S3**
   - Location: `s3://my-bucket/synthea/raw/{table_name}.csv`

2. **S3 Event Notification** triggers on `PUT` event

3. **AWS Lambda** extracts the table name from file path and triggers this notebook via **Databricks Jobs API**
   - Payload:
     ```json
     {
       "notebook_params": {
         "table_name": "patients"
       }
     }
     ```

4. **Databricks Job runs this notebook**, ingesting and cleaning the specific file, then saving as a Delta table in the Bronze layer.

---

##  Simulated Job Input
Below, we use `dbutils.widgets` to simulate workflow input:

Step 1: Set Workflow Parameters

In [0]:
# For workflow use: receive param from a job
dbutils.widgets.text("table_name", "")  # Empty by default
param_table = dbutils.widgets.get("table_name")

Step 2: Definte Reusable Functions

In [0]:
from pyspark.sql.functions import current_timestamp, lit
import re

# Clean column names using a consistent pattern
def clean_column_name(col_name):
    return re.sub(r'[^a-zA-Z0-9]', '_', col_name).lower().strip('_')

# Process a single CSV into a Delta-formatted Bronze table
def process_bronze_table(table_name: str, source_dir="/FileStore/tables", bronze_dir="/FileStore/bronze"):
    source_path = f"{source_dir}/{table_name}.csv"
    bronze_path = f"{bronze_dir}/{table_name}"
    bronze_table = f"bronze_{table_name}"

    print(f"Processing table: {table_name}")

    # Load CSV
    df = (
        spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(source_path)
    )

    # Clean column names
    df = df.toDF(*[clean_column_name(c) for c in df.columns])

    # Add metadata columns
    df = df.withColumn("ingestion_timestamp", current_timestamp()) \
           .withColumn("source_file", lit(source_path))

    # Write Delta file to Bronze path
    df.write.format("delta").mode("overwrite").save(bronze_path)

    # Register the Delta file as a Hive table
    spark.sql(f"DROP TABLE IF EXISTS {bronze_table}")
    spark.sql(f"CREATE TABLE {bronze_table} USING DELTA LOCATION '{bronze_path}'")

    print(f"Bronze table created: {bronze_table}")


Step 3: Process Table(s)

In [0]:
bronze_tables = ["patients", "encounters", "providers", "conditions", "organizations"]

# Decide: run one table (workflow) or all (manual)
if param_table:
    process_bronze_table(param_table)
else:
    for table in bronze_tables:
        process_bronze_table(table)


Processing table: patients
Bronze table created: bronze_patients
Processing table: encounters
Bronze table created: bronze_encounters
Processing table: providers
Bronze table created: bronze_providers
Processing table: conditions
Bronze table created: bronze_conditions
Processing table: organizations
Bronze table created: bronze_organizations
